# English Literature Analysis

## Data Preparation / Data Cleaning

### Tasks

1. Read all the sheets in excel file(input) to a list of dataframes.
2. Standardize the column names.
3. Match the names in different sheets and update corresponding addresses.
 * Mark the conflicts in a different column.
4. Merge London city and non-London city latitudes and longitudes.
5. Update the latitudes and longitudes of entries with same addresses.
6. Find the missing latitudes and longitudes for the addresses using API.
7. Remove/merge unnecessary columns and create final dataframe.
8. Write the final dataframe into excel.

### Libraries

In [1]:
import sys
import os.path
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # Required to find latitudes and longitudes of places
from pandas import ExcelWriter

### Important constants

In [2]:
INPUT_FILE_NAME = 'data/Network Data Lists-7-14.xlsx'
OUTPUT_FILE_NAME = 'data/Final_dataset.xlsx'
INPUT_COLUMNS = ['Name', 'List_ID', 'Year', 'Reformatted_Name','Gender', 'is_from_London', 'London_Street',
                 'London_Region', 'London_Lat_Long', 'City','Non_London_City_Lat_Long', 'Nation']
INTERMEDIATE_COLUMNS = ['Name', 'List_ID', 'Year', 'Reformatted_Name','Gender', 'is_from_London', 'London_Street',
                 'London_Region', 'London_Lat_Long', 'City','Non_London_City_Lat_Long', 'Nation', 'Conflits', 'location_update_from_other_sheet', 'coordinates']
OUTPUT_COLUMNS = ['Name', 'Reformatted_Name', 'List_ID', 'Year','Gender', 'is_from_London', 'London_Street',
                 'London_Region', 'City', 'Nation', 'Lat_Long', 'Conflits']
SHEETS = None

### 1. Read all the sheets in excel file(input) to a list of dataframes.

In [3]:
# Returns list of dataframes
def read_input():
    global SHEETS
    df_list = []
    xl = pd.ExcelFile(INPUT_FILE_NAME)
    SHEETS = xl.sheet_names[:-1]        # Remove last sheet(Project Details)
    print("Sheets in dataframe")
    print(SHEETS)
    for i in range(len(SHEETS)):
        df = xl.parse(SHEETS[i])
        df_list.append(df)
    return df_list

### 2. Standardize the column names.

In [4]:
def standardize_column_names(df_list):
    required_columns_df_list = []
    extra_columns_df_list = []
    for df in df_list:
        df.columns = INPUT_COLUMNS + list(df.columns)[len(INPUT_COLUMNS):]
        required_columns_df_list.append(df.loc[:,INPUT_COLUMNS])
        extra_columns_df_list.append(df.loc[:,list(df.columns)[len(INPUT_COLUMNS):]])
    # check_column_names(required_columns_df_list)
    # check_column_names(extra_columns_df_list)
    return required_columns_df_list, extra_columns_df_list

# Utility function to check column names
def check_column_names(df_list):
    for df in df_list:
        print(df.columns[:len(INPUT_COLUMNS)])

### 3. Match the names in different sheets and update corresponding addresses.

In [5]:
# This will add two new columns:
# 1. 'location_update_from_other_sheet' : indicating whether the location entry was updated from other sheets
# 2. 'Conflicts' : Indicates possible locations (Same name different addresses)

def add_columns(df_list):
    for df in df_list:
        df['location_update_from_other_sheet'] = ''
        df['Conflits'] = ''
    return df_list

def get_indexes_of_missing_address(df):
    indexes = df[df['London_Street'].isnull() | df['London_Region'].isnull() | df['City'].isnull()].index.tolist()
    return indexes

def get_indexes_of_matching_names(df, name):
    indexes = df[df['Reformatted_Name'] == name].index.tolist()
    return indexes

def update_row_col(df, row, col, val, matching_sheet_idx, matchin_row_idx):
    if val != 'nan' and val != '0':
        if str(df.loc[row, col]) == 'nan':
                df.loc[row, col] = val
                df.loc[row, 'location_update_from_other_sheet'] = 'Yes, Sheet:' + str(matching_sheet_idx) + ' row:' + str(matchin_row_idx)
        elif df.loc[row, col] != val and df.loc[row, col]!=0:
            df.loc[row, 'Conflits'] += ' '+col+' : '+val+ ' Sheet:' + str(matching_sheet_idx) + ' row:' + str(matchin_row_idx) + ','

def update_address_crosscheck_df(df_list, df_idx):
    missing_rows_indexes = get_indexes_of_missing_address(df_list[df_idx])
    for missing_idx in missing_rows_indexes:  # Iterating over missing rows
        for i in range(len(df_list)): # Iterating over other sheets(df)
            if i != df_idx:           # Check for self sheet
                matching_row_indexes = get_indexes_of_matching_names(df_list[i], df_list[df_idx].loc[missing_idx,'Reformatted_Name'])
                for matching_idx in matching_row_indexes:
                    update_row_col(df_list[df_idx], missing_idx, 'London_Street', str(df_list[i].loc[matching_idx, 'London_Street']), i, matching_idx)
                    update_row_col(df_list[df_idx], missing_idx, 'London_Region', str(df_list[i].loc[matching_idx, 'London_Region']), i, matching_idx)
                    update_row_col(df_list[df_idx], missing_idx, 'City', str(df_list[i].loc[matching_idx, 'City']), i, matching_idx)

def update_address_crosscheck(df_list):
    df_list = add_columns(df_list)
    for i in range(len(df_list)):
        update_address_crosscheck_df(df_list, i)

### 4. Merge London city and non-London city latitudes and longitudes

In [6]:
# Creates a new column 'coordinates' which has latitude and longitude of any address
def merge_lat_long(df_list):
    for df in df_list:
        df['coordinates'] = np.nan
        df.loc[df[df['London_Lat_Long'].notnull() & df['London_Lat_Long']!=0].index, 'coordinates'] = df[df['London_Lat_Long'].notnull() & df['London_Lat_Long']!=0]['London_Lat_Long']
        df.loc[df[df['Non_London_City_Lat_Long'].notnull() & df['Non_London_City_Lat_Long']!=0].index, 'coordinates'] = df[df['Non_London_City_Lat_Long'].notnull() & df['Non_London_City_Lat_Long']!=0]['Non_London_City_Lat_Long']

### 5. Update the latitudes and longitudes of entries with same addresses.

In [7]:
required_columns = ['London_Street', 'London_Region', 'City', 'Nation', 'coordinates']

'''def get_unique_df(df_list):
    unique_df = pd.DataFrame()
    for df in df_list:
        unique_df = unique_df.append(df[df['coordinates'].notnull()][required_columns].drop_duplicates(subset=required_columns[:-1]))
    return unique_df

def update_lat_lon_with_same_address(df_list):
    unique_df = get_unique_df(df_list)
    #print(unique_df.describe())
    for df in df_list:
        print('-------------')
        print(sum(pd.merge(df, unique_df, on=required_columns[:-1], indicator=True))
        #print(df[required_columns].loc[:,required_columns[:-1]].columns)
        #print(unique_df.loc[:,required_columns[:-1]].columns)
        #print(df[required_columns].loc[:,required_columns[:-1]]==unique_df.loc[:,required_columns[:-1]])'''

def update_lat_lon_with_same_address(df_list):
    large_df = pd.concat(dict(enumerate(df_list)), ignore_index=False)
    for col in large_df:
        large_df[col] = ['' if (not isinstance(val, str) and np.isnan(val)) else 
                   (val if isinstance(val, str) else str(int(val))) 
                   for val in large_df[col].tolist()]
    large_df['coordinates'] = large_df.groupby(required_columns[:-1])['coordinates'].transform(lambda x: x.dropna().iloc[0] if x.notnull().any() else np.nan)
    tmp_list = [x.reset_index(level=0, drop=True) for i, x in large_df.groupby(level=0)]
    return tmp_list

### 6. Find the missing latitudes and longitudes for the addresses using API.

In [8]:
geolocator = Nominatim()
def get_location(street, region, city, nation):
    try:
        location = geolocator.geocode(street+' '+region+' '+ city+' '+nation, timeout=5)
        if location == None:
            location = geolocator.geocode(street+' '+city+' '+nation, timeout=5)
            if location == None:
                location = geolocator.geocode(region+' '+city+' '+nation, timeout=5)
                if location == None:
                    location = geolocator.geocode(city+' '+nation, timeout=5)
                    if location == None:
                        location = geolocator.geocode(nation, timeout=5)
        return location
    except Exception:
        print(street, region, city, nation)
        print(Exception.args)
        return None

def make_key(row):
    return str(row['London_Street']) + str(row['London_Region']) + str(row['City']) + str(row['Nation'])

def update_missing_coordinates_with_geoapi(df_list):
    LOCATION_COLUMNS = ['London_Street', 'London_Region', 'City', 'Nation']
    i = 0
    for df in df_list:
        new_df = df[df['coordinates']==''].loc[:,LOCATION_COLUMNS].dropna(axis=0, how='all', subset=LOCATION_COLUMNS)
        print('Processing: ', SHEETS[i])
        i += 1
        for idx in new_df.index:
            if df.loc[idx,'coordinates'] == '':
                street = ''
                region = ''
                city = ''
                nation = ''
                if not pd.isnull(new_df.loc[idx,'London_Street']) and new_df.loc[idx,'London_Street']!='0':
                    street = new_df.loc[idx,'London_Street']
                if not pd.isnull(new_df.loc[idx,'London_Region']) and new_df.loc[idx,'London_Region']!='0':
                    region = new_df.loc[idx,'London_Region']
                if not pd.isnull(new_df.loc[idx,'City']) and new_df.loc[idx,'City']!="0":
                    city = new_df.loc[idx,'City']
                if not pd.isnull(new_df.loc[idx,'Nation']) and new_df.loc[idx,'Nation']!='0':
                    nation = new_df.loc[idx,'Nation']
                location = get_location(street, region, city, nation)
                #location = str(street) + str(region) + str(city) + str(nation)
                
                if location != None:
                    #df.loc[df[df.apply(make_key,axis=1)==make_key(df.iloc[idx])].index.tolist(), 'coordinates'] = str(location)
                    df.loc[df[df.apply(make_key,axis=1)==make_key(df.iloc[idx])].index.tolist(), 'coordinates'] = str(location.latitude) + ", " + str(location.longitude)

### 7. Remove/merge unnecessary columns and create final dataframe.

In [9]:
def create_final_dataframe(df_list, extra_columns_df_list):
    new_df = []
    for i in range(len(df_list)):
        new_df.append(pd.concat([df_list[i],extra_columns_df_list[i]], axis=1))
    return new_df

### 8. Write the final dataframe into excel.

In [10]:
def save_xls(df_list):
    global SHEETS
    writer = ExcelWriter(OUTPUT_FILE_NAME)
    for i in range(len(SHEETS)):
        df_list[i].to_excel(writer,SHEETS[i])
    writer.save()

## Client

In [11]:
df_list = read_input()

Sheets in dataframe
['Emma Lyon 1812', 'Bolaffey 1820', 'Polack 1830', ' Moss 1839', ' Infant 1841', ' VOJ 1842-45', 'Belisario 1856']


In [12]:
df_list, extra_columns_df_list = standardize_column_names(df_list)
#check_column_names(df_list)

In [13]:
update_address_crosscheck(df_list)

In [14]:
merge_lat_long(df_list)

In [15]:
df_list = update_lat_lon_with_same_address(df_list)

In [16]:
update_missing_coordinates_with_geoapi(df_list)

In [17]:
df_list = create_final_dataframe(df_list, extra_columns_df_list)

In [18]:
save_xls(df_list)